# Day 10: Factory

## Part 1

In [ ]:
with open('input.txt', 'r') as f:
    data = f.read().split('\n')[:-1]

In [ ]:
machines = []
for row in data:
    split_row = row.split(' ')
    diagram = [1 if i == '#' else 0 for i in split_row[0].replace('[', '').replace(']', '')]
    buttons = []
    for button in split_row[1:-1]:
        buttons.append([int(i) for i in button.replace('(', '').replace(')', '').split(',')])

    joltage = [int(i) for i in split_row[-1].replace('{', '').replace('}', '').split(',')]

    machines.append([diagram, buttons, joltage])

In [ ]:
def get_potential_buttons(diagram, buttons):
    needed_indices = []
    for i in range(len(diagram)):
        if(diagram[i] == 1):
            needed_indices.append(i)

    needed_buttons = {}
    for button in buttons:
        num_affected = 0

        for i in button:
            if(i in needed_indices):
                num_affected += 1

        if(num_affected > 0):
            needed_buttons[tuple(button)] = num_affected

    return {k:v for k, v in sorted(needed_buttons.items(), key=lambda item: item[1], reverse=True)}

def press_button(state, button):
    new_state = []
    for i in range(len(state)):
        if(i in button):
            new_state.append((state[i]+1)%2)
        else:
            new_state.append(state[i])

    return new_state

ops = set()
prev_states = {}
solution_buttons = {}
def configure(diagram, state, buttons, depth=0):
    global ops
    global prev_states

    if(depth == 0):
        ops = set()
        prev_states = {}
    elif((len(ops) > 0 and depth >= min(ops)) or depth > len(diagram)**2):
        return -1

    if(tuple(state) in prev_states and prev_states.get(tuple(state)) < depth):
        return -1
    else:
        prev_states[tuple(state)] = depth

    if(diagram == state):
        return depth
    
    needed_operations = []
    for i in range(len(diagram)):
        if(diagram[i] != state[i]):
            needed_operations.append(1)
        else:
            needed_operations.append(0)

    potential_buttons = get_potential_buttons(needed_operations, buttons)
    for button in potential_buttons:
        new_state = press_button(state, button)
        solved_depth = configure(diagram, new_state, buttons, depth+1)

        if(solved_depth > 0 and solved_depth not in ops):
            ops.add(solved_depth)
            if(solved_depth == 1):
                solution_buttons[(tuple(diagram),tuple(button))] = solved_depth
    
    return -1 if ops == set() else min(ops)


total = 0
for index, machine in enumerate(machines):
    diagram = machine[0]
    buttons = machine[1]
    joltage = machine[2] # "can be safely ignored" lol

    cur_state = [0 for i in range(len(diagram))]
    total += configure(diagram, cur_state, buttons)
    # print(f'{index}/{len(machines)}')

# print(total)

## Part 2